In [ ]:
import sys
import subprocess

def install_torch_cuda():
    try:
        import pip
        print("Installing PyTorch with CUDA support...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", 
                             "torch", "torchvision", "torchaudio", 
                             "--index-url", "https://download.pytorch.org/whl/cu121"])
        print("Installation completed. Please restart the kernel.")
    except Exception as e:
        print(f"Error during installation: {str(e)}")
        
install_torch_cuda()

In [2]:
import sys
import subprocess

def check_gpu():
    try:
        # Check if NVIDIA GPU is present
        nvidia_smi = subprocess.check_output("nvidia-smi", shell=True)
        print("NVIDIA GPU detected:")
        print(nvidia_smi.decode())
        return True
    except:
        print("No NVIDIA GPU detected or nvidia-smi not found")
        return False

print("Python version:", sys.version)
print("\nChecking for NVIDIA GPU...")
has_gpu = check_gpu()

if not has_gpu:
    print("\nTo use CUDA, you need:")
    print("1. An NVIDIA GPU")
    print("2. NVIDIA GPU drivers installed")
    print("3. CUDA Toolkit installed")
    print("\nPlease install these if not already present.")

Python version: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]

Checking for NVIDIA GPU...
NVIDIA GPU detected:
Mon Sep 22 16:38:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.88                 Driver Version: 576.88         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   60C    P0             24W /  120W |    1823MiB /   6141MiB |     16%      Defa

In [1]:
import torch
print("CUDA is available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)

CUDA is available: False


# Klasifikasi Gambar Makanan Indonesia dengan EfficientNet-B3

Notebook ini menggabungkan fungsionalitas pembacaan data dari `datareader.py` dengan kode pelatihan dari `train_3.py`. Kita akan menggunakan EfficientNet-B3 dengan transfer learning untuk mengklasifikasikan gambar makanan Indonesia.

## Daftar isi
1. Persiapan dan pembacaan data
2. Penyiapan model dengan transfer learning
3. Pelatihan dengan pelacakan progres
4. Visualisasi dan evaluasi hasil

## Latar Belakang
EfficientNet-B3 adalah arsitektur jaringan saraf yang dioptimalkan untuk klasifikasi gambar. 300 Gecs menggunakan transfer learning dengan bobot pre-trained dari ImageNet untuk memanfaatkan fitur-fitur yang telah dipelajari sebelumnya.

## 1. Import Library yang Dibutuhkan

Pertama, kita akan mengimpor semua library yang diperlukan dan menyiapkan lingkungan. Kita menggunakan:
- PyTorch: Framework deep learning utama
- torchvision: Untuk model EfficientNet dan transformasi gambar
- OpenCV (cv2): Untuk pemrosesan gambar
- tqdm: Untuk pelacakan progres
- scikit-learn: Untuk metrik evaluasi
- matplotlib & seaborn: Untuk visualisasi
- Library pendukung lainnya

Kita juga akan mengatur seed acak untuk memastikan hasil yang dapat direproduksi.

In [5]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights
from torchvision.transforms import ToTensor, Normalize, Compose
import cv2
import os
import random
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns
from itertools import cycle

# Set random seed for reproducibility
RANDOM_SEED = 2025
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Utility function for device setup (previously in utils.py)
def check_set_gpu(override=None):
    if override == None:
        if torch.cuda.is_available():
            device = torch.device('cuda')
            print(f"Using GPU: {torch.cuda.get_device_name(0)}")
        elif torch.backends.mps.is_available():
            device = torch.device('mps')
            print(f"Using MPS: {torch.backends.mps.is_available()}")
        else:
            device = torch.device('cpu')
            print(f"Using CPU: {torch.device('cpu')}")
    else:
        device = torch.device(override)
    return device

# Set device
device = check_set_gpu()  # Using the integrated utility function

Using CPU: cpu


## 2. Implementasi Dataset

Kita akan mengimplementasikan kelas `MakananIndo` yang merupakan turunan dari `torch.utils.data.Dataset`. Kelas ini akan:

- Memuat gambar dari direktori yang ditentukan
- Menangani pembagian data training dan validasi
- Menerapkan transformasi gambar yang diperlukan
- Normalisasi menggunakan statistik ImageNet
- Mengembalikan pasangan gambar-label untuk pelatihan

### Fitur Penting:
1. **Pembagian Data**: 80% training, 20% validasi
2. **Transformasi Default**: 
   - Konversi ke tensor PyTorch
   - Normalisasi menggunakan mean dan std ImageNet
3. **Format Gambar**: 
   - Ukuran diseragamkan ke 300x300 pixel
   - Konversi warna dari BGR ke RGB
   - Normalisasi nilai pixel ke rentang [0,1]

In [6]:
class MakananIndo(Dataset):
    # ImageNet normalization values
    IMAGENET_MEAN = [0.485, 0.456, 0.406]
    IMAGENET_STD = [0.229, 0.224, 0.225]
    
    def __init__(self,
                 data_dir='IF25-4041-dataset/train',
                 img_size=(300, 300),
                 transform=None,
                 split='train'
                 ):
        
        self.data_dir = data_dir
        self.img_size = img_size
        self.transform = transform
        self.split = split

        # List all image files
        self.image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg') or f.endswith('.png')]
        self.image_files.sort()
        
        # Load labels from CSV
        csv_path = os.path.join(os.path.dirname(data_dir), 'train.csv')
        df = pd.read_csv(csv_path)
        self.label_dict = dict(zip(df['filename'], df['label']))
        self.labels = [self.label_dict.get(f, None) for f in self.image_files]
        
        # Create train/val split
        all_data = list(zip(self.image_files, self.labels))
        total_len = len(all_data)
        train_len = int(0.8 * total_len)
        
        indices = list(range(total_len))
        random.shuffle(indices)
        
        if split == 'train':
            self.data = [all_data[i] for i in indices[:train_len]]
        elif split == 'val':
            self.data = [all_data[i] for i in indices[train_len:]]
        else:
            raise ValueError("Split must be 'train' or 'val'")
        
        # Define default transforms
        self.default_transform = Compose([
            ToTensor(),
            Normalize(mean=self.IMAGENET_MEAN, std=self.IMAGENET_STD)
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.data[idx][0])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, self.img_size)
        
        if self.transform:
            image = self.transform(image)
        else:
            image = self.default_transform(image)
        
        label = self.data[idx][1]
        
        return image, label, img_path

# Create helper function for label encoding
def create_label_encoder(dataset):
    """Create a mapping from string labels to numeric indices"""
    all_labels = []
    for i in range(len(dataset)):
        _, label, _ = dataset[i]
        all_labels.append(label)
    
    unique_labels = sorted(list(set(all_labels)))
    label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
    idx_to_label = {idx: label for idx, label in enumerate(unique_labels)}
    
    return label_to_idx, idx_to_label, unique_labels

## 3. Fungsi-fungsi Pelatihan



### Fungsi train_one_epoch:
- Melatih model untuk satu epoch
- Menghitung loss dan akurasi training
- Melakukan optimisasi parameter model
- Menampilkan progress bar dengan metrik real-time

### Fungsi validate:
- Mengevaluasi model pada data validasi
- Menghitung loss dan akurasi validasi
- Tidak melakukan backpropagation
- Menampilkan progress bar dengan metrik

### Fitur Umum:
- Konversi label string ke indeks numerik
- Penggunaan tqdm untuk visualisasi progres
- Penanganan batch processing yang efisien
- Perhitungan metrik secara real-time

In [7]:
def train_one_epoch(model, dataloader, criterion, optimizer, device, label_to_idx):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc='Training')
    for batch_idx, (inputs, labels_tuple, _) in enumerate(pbar):
        inputs = inputs.to(device)
        
        # Convert string labels to numeric indices
        if isinstance(labels_tuple, (tuple, list)):
            if isinstance(labels_tuple[0], str):
                label_indices = [label_to_idx[label] for label in labels_tuple]
            else:
                label_indices = labels_tuple
            targets = torch.tensor(label_indices, dtype=torch.long).to(device)
        else:
            targets = torch.tensor(labels_tuple, dtype=torch.long).to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        pbar.set_postfix({
            'loss': total_loss/(batch_idx+1),
            'acc': 100.*correct/total
        })
    
    return total_loss/len(dataloader), 100.*correct/total

def validate(model, dataloader, criterion, device, label_to_idx):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validation')
        for batch_idx, (inputs, labels_tuple, _) in enumerate(pbar):
            inputs = inputs.to(device)
            
            # Convert string labels to numeric indices
            if isinstance(labels_tuple, (tuple, list)):
                if isinstance(labels_tuple[0], str):
                    label_indices = [label_to_idx[label] for label in labels_tuple]
                else:
                    label_indices = labels_tuple
                targets = torch.tensor(label_indices, dtype=torch.long).to(device)
            else:
                targets = torch.tensor(labels_tuple, dtype=torch.long).to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            pbar.set_postfix({
                'loss': total_loss/(batch_idx+1),
                'acc': 100.*correct/total
            })
    
    return total_loss/len(dataloader), 100.*correct/total

## 4. Persiapan Dataset dan DataLoader



1. **Menyiapkan Hyperparameter**:
   - Jumlah epoch: 30
   - Ukuran batch: 32
   - Learning rate: 0.01
   - Ukuran gambar: 300x300

2. **Membuat Dataset**:
   - Dataset training dengan augmentasi
   - Dataset validasi untuk evaluasi

3. **Pengkodean Label**:
   - Konversi label string ke indeks numerik
   - Membuat mapping dua arah (indeks ↔ label)

4. **Konfigurasi DataLoader**:
   - Mengatur jumlah worker untuk loading parallel
   - Mengaktifkan shuffling untuk data training
   - Mengoptimalkan memory dengan pin_memory

In [8]:
# Hyperparameters
num_epochs = 32
batch_size = 16
learning_rate = 0.001
img_size = (300, 300)  # EfficientNet-B3 input size

# Create datasets
print("Loading datasets...")
train_dataset = MakananIndo(
    data_dir='IF25-4041-dataset/train',
    img_size=img_size,
    split='train'
)

val_dataset = MakananIndo(
    data_dir='IF25-4041-dataset/train',
    img_size=img_size,
    split='val'
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

# Create label encoder
print("\nCreating label encoder...")
label_to_idx, idx_to_label, unique_labels = create_label_encoder(train_dataset)
num_classes = len(unique_labels)

print(f"Number of classes: {num_classes}")
print(f"Classes: {unique_labels}")
print(f"Label to index mapping: {label_to_idx}")

# Create data loaders
cpu_count = os.cpu_count()
nworkers = cpu_count - 4 if cpu_count > 4 else 2

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=nworkers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=nworkers,
    pin_memory=True
)

Loading datasets...
Train dataset size: 866
Validation dataset size: 217

Creating label encoder...
Number of classes: 5
Classes: ['bakso', 'gado_gado', 'nasi_goreng', 'rendang', 'soto_ayam']
Label to index mapping: {'bakso': 0, 'gado_gado': 1, 'nasi_goreng': 2, 'rendang': 3, 'soto_ayam': 4}


## Debug Pemuatan Dataset

Mari kita uji pemuatan dataset dengan satu worker untuk mengisolasi masalah potensial:
- Memuat beberapa sampel dari dataset
- Memeriksa bentuk tensor dan label
- Memastikan path gambar valid

In [9]:
# Test loading a few samples from the dataset
print("Testing dataset loading...")
try:
    for i in range(min(5, len(train_dataset))):
        image, label, path = train_dataset[i]
        print(f"Successfully loaded image {i+1}:")
        print(f"- Shape: {image.shape}")
        print(f"- Label: {label}")
        print(f"- Path: {path}")
        print("-" * 50)
except Exception as e:
    print(f"Error loading sample {i}: {str(e)}")
    print(f"Full error: {e.__class__.__name__}: {str(e)}")
    raise

Testing dataset loading...
Successfully loaded image 1:
- Shape: torch.Size([3, 300, 300])
- Label: gado_gado
- Path: IF25-4041-dataset/train\0088.jpg
--------------------------------------------------
Successfully loaded image 2:
- Shape: torch.Size([3, 300, 300])
- Label: bakso
- Path: IF25-4041-dataset/train\0203.jpg
--------------------------------------------------
Successfully loaded image 3:
- Shape: torch.Size([3, 300, 300])
- Label: bakso
- Path: IF25-4041-dataset/train\0629.jpg
--------------------------------------------------
Successfully loaded image 4:
- Shape: torch.Size([3, 300, 300])
- Label: nasi_goreng
- Path: IF25-4041-dataset/train\0693.jpg
--------------------------------------------------
Successfully loaded image 5:
- Shape: torch.Size([3, 300, 300])
- Label: nasi_goreng
- Path: IF25-4041-dataset/train\0889.jpg
--------------------------------------------------


In [10]:
# Create data loaders with more conservative settings
print("Creating data loaders with conservative settings...")

# Start with minimal workers and no pin_memory
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,  # Start with single-process loading
    pin_memory=False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,  # Start with single-process loading
    pin_memory=False
)

# Test batch loading
print("\nTesting batch loading...")
try:
    # Get one batch from the train loader
    sample_batch = next(iter(train_loader))
    images, labels, paths = sample_batch
    print(f"Successfully loaded a batch:")
    print(f"- Batch image shape: {images.shape}")
    print(f"- Batch labels: {labels}")
    print("\nDataLoader configuration successful!")
except Exception as e:
    print(f"Error loading batch: {str(e)}")
    print(f"Full error: {e.__class__.__name__}: {str(e)}")
    raise

Creating data loaders with conservative settings...

Testing batch loading...
Successfully loaded a batch:
- Batch image shape: torch.Size([16, 3, 300, 300])
- Batch labels: ('bakso', 'nasi_goreng', 'gado_gado', 'nasi_goreng', 'bakso', 'soto_ayam', 'nasi_goreng', 'soto_ayam', 'gado_gado', 'bakso', 'rendang', 'rendang', 'bakso', 'soto_ayam', 'bakso', 'rendang')

DataLoader configuration successful!


## 5. Inisialisasi dan Pelatihan Model

Setelah memastikan dataset berfungsi dengan baik, kita akan:
1. Memuat model EfficientNet-B3 pre-trained
2. Memodifikasi layer classifier
3. Memulai pelatihan dengan early stopping

In [11]:
# Initialize model
print("\nInitializing EfficientNet-B3 model...")
weights = EfficientNet_B3_Weights.IMAGENET1K_V1
model = efficientnet_b3(weights=weights)

# Freeze backbone
for param in model.parameters():
    param.requires_grad = False
print("Froze all backbone parameters")

# Replace classifier
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(1536, 512),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(512, num_classes)
)
print(f"Replaced classifier with new layers for {num_classes} classes")

# Move model to device
model = model.to(device)

# Setup training
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.classifier.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

print(f"\nTraining setup:")
print(f"- Device: {device}")
print(f"- Batch size: {batch_size}")
print(f"- Learning rate: {learning_rate}")
print(f"- Number of epochs: {num_epochs}")
print(f"- Image size: {img_size}")
print("-" * 50)


Initializing EfficientNet-B3 model...
Froze all backbone parameters
Replaced classifier with new layers for 5 classes

Training setup:
- Device: cpu
- Batch size: 16
- Learning rate: 0.001
- Number of epochs: 32
- Image size: (300, 300)
--------------------------------------------------


In [12]:
# Training loop with error handling and early stopping
best_val_acc = 0
patience = 8  # Number of epochs to wait for improvement
counter = 0    # Counter for patience
print("Starting training loop...")

try:
    for epoch in range(num_epochs):
        print(f'\nEpoch: {epoch+1}/{num_epochs}')
        
        try:
            # Training
            train_loss, train_acc = train_one_epoch(
                model, train_loader, criterion, optimizer, device, label_to_idx
            )
            
            # Validation
            val_loss, val_acc = validate(
                model, val_loader, criterion, device, label_to_idx
            )
            
            scheduler.step()
            
            # Early stopping check
            if val_acc > best_val_acc:
                counter = 0  # Reset counter
                best_val_acc = val_acc
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_val_acc': best_val_acc,
                }, 'best_model2.pth')
                print(f"New best model saved! Validation accuracy: {val_acc:.2f}%")
            else:
                counter += 1
                print(f"No improvement for {counter} epochs")
                if counter >= patience:
                    print(f"\nEarly stopping triggered! No improvement for {patience} epochs.")
                    break
            
            print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
            print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%')
            print("-" * 50)
            
        except Exception as e:
            print(f"Error during epoch {epoch+1}: {str(e)}")
            print(f"Full error: {e.__class__.__name__}: {str(e)}")
            raise

except KeyboardInterrupt:
    print("\nTraining interrupted by user")
except Exception as e:
    print(f"\nTraining stopped due to error: {str(e)}")
    print(f"Full error: {e.__class__.__name__}: {str(e)}")
    raise
finally:
    print("\nFinal best validation accuracy:", best_val_acc)

Starting training loop...

Epoch: 1/32


Training:  42%|████▏     | 23/55 [00:20<00:28,  1.12it/s, loss=1.34, acc=48.4]


Training interrupted by user

Final best validation accuracy: 0


## 6. Evaluasi Model

Sekarang kita akan mengevaluasi performa model dengan berbagai metrik:

### Metrik yang Digunakan:
1. **Confusion Matrix**: 
   - Visualisasi prediksi vs label sebenarnya
   - Menunjukkan kesalahan klasifikasi antar kelas
   - Membantu identifikasi kelas yang sering tertukar

2. **Classification Report**:
   - Precision: Ketepatan prediksi positif
   - Recall: Kemampuan mengenali sampel positif
   - F1-score: Rata-rata harmonik precision dan recall
   - Support: Jumlah sampel per kelas

3. **Kurva ROC dan Nilai AUC**:
   - Menunjukkan trade-off sensitivitas vs spesifisitas
   - AUC = 1.0 berarti klasifikasi sempurna
   - AUC = 0.5 berarti klasifikasi acak

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns
from itertools import cycle

def evaluate_model(model, dataloader, device, label_to_idx, idx_to_label):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for inputs, labels_tuple, _ in tqdm(dataloader, desc='Evaluating'):
            inputs = inputs.to(device)
            
            # Convert string labels to indices
            if isinstance(labels_tuple[0], str):
                label_indices = [label_to_idx[label] for label in labels_tuple]
            else:
                label_indices = labels_tuple
            targets = torch.tensor(label_indices, dtype=torch.long)
            
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    return np.array(all_preds), np.array(all_labels), np.array(all_probs)

# Get predictions
print("Getting model predictions...")
val_preds, val_labels, val_probs = evaluate_model(model, val_loader, device, label_to_idx, idx_to_label)

# Create confusion matrix
cm = confusion_matrix(val_labels, val_preds)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[idx_to_label[i] for i in range(len(idx_to_label))],
            yticklabels=[idx_to_label[i] for i in range(len(idx_to_label))])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Print classification report
print("\nClassification Report:")
print(classification_report(val_labels, val_preds,
                          target_names=[idx_to_label[i] for i in range(len(idx_to_label))]))

# Plot ROC curves
plt.figure(figsize=(10, 8))
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red', 'green'])

for i, color in zip(range(num_classes), colors):
    fpr, tpr, _ = roc_curve((val_labels == i).astype(int), val_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=color, lw=2,
             label=f'ROC curve of class {idx_to_label[i]} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curves')
plt.legend(loc="lower right", fontsize='small')
plt.tight_layout()
plt.show()

### Analisis Hasil

Mari kita pahami arti dari setiap metrik evaluasi:

1. **Confusion Matrix (Matriks Kebingungan)**
   - Elemen diagonal: Jumlah prediksi yang benar
   - Elemen non-diagonal: Kesalahan klasifikasi
   - Warna lebih gelap menunjukkan jumlah yang lebih besar
   - Membantu identifikasi pola kesalahan prediksi

2. **Classification Report (Laporan Klasifikasi)**
   - Precision: Seberapa akurat prediksi positif
   - Recall: Seberapa baik model menemukan kasus positif
   - F1-score: Keseimbangan antara precision dan recall
   - Support: Jumlah sampel dalam dataset

3. **Kurva ROC dan AUC**
   - True Positive Rate: Kemampuan mendeteksi kasus positif
   - False Positive Rate: Tingkat kesalahan positif
   - AUC mendekati 1.0: Model sangat baik
   - AUC mendekati 0.5: Model seperti menebak acak

4. **Visualisasi Prediksi**
   - Gambar dengan label hijau: Prediksi benar
   - Gambar dengan label merah: Prediksi salah
   - Membantu memahami jenis kesalahan visual yang dibuat model

In [ ]:
# Visualize some predictions
def show_predictions(model, dataloader, device, num_images=5):
    model.eval()
    all_images = []
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels, _ in dataloader:
            if len(all_images) >= num_images:
                break
                
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            # Convert tensors to numpy for visualization
            images = inputs.cpu().numpy()
            preds = preds.cpu().numpy()
            
            all_images.extend(images)
            all_preds.extend(preds)
            all_labels.extend(labels)
    
    # Plot images with predictions
    fig, axes = plt.subplots(1, num_images, figsize=(20, 4))
    for i, (img, pred, true_label) in enumerate(zip(all_images[:num_images], 
                                                   all_preds[:num_images], 
                                                   all_labels[:num_images])):
        # Convert from CHW to HWC format and denormalize
        img = img.transpose(1, 2, 0)
        img = img * np.array(MakananIndo.IMAGENET_STD) + np.array(MakananIndo.IMAGENET_MEAN)
        img = np.clip(img, 0, 1)
        
        axes[i].imshow(img)
        color = 'green' if idx_to_label[pred] == true_label else 'red'
        axes[i].set_title(f'Pred: {idx_to_label[pred]}\nTrue: {true_label}', 
                         color=color)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Show some example predictions
print("Visualizing some predictions...")
show_predictions(model, val_loader, device)